In [14]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np

%matplotlib notebook

In [15]:
class Model(object):
    def __init__(self, train_acc_vs_t_function, init_train_duration, auto_checkpoint = False):
        self.train_fn = train_acc_vs_t_function
        self.init_trained_duration = init_train_duration
        self.init_acc = self.train_fn(self.init_trained_duration)
        self.auto_checkpoint = auto_checkpoint
        self.reset()

    def post_train_acc(self, train_time):
        return self.train_fn(train_time + self.init_trained_duration)
    
    def train(self, train_time):
        self.trained_duration += train_time
        self.uncheckpointed_acc = self.train_fn(self.trained_duration)
        if self.auto_checkpoint:
            self.checkpoint()
    
    def checkpoint(self):
        self.acc = self.uncheckpointed_acc
    
    def delta_acc(self, t):
        return self.post_train_acc(t + 1) - self.post_train_acc(t)
    
    def reset(self):
        self.trained_duration = self.init_trained_duration
        self.acc = self.init_acc
        self.uncheckpointed_acc = self.acc


def slowed_acc(acc, contention_slowdown = 0.9):
    return acc*contention_slowdown

def optimus_fn(t, T, scale):
    if t==0:
        return 0
    else:
        return scale*1/((1/(t*10*1/T)) + 1) * 100   # Hits 0.9 acc at time T. *100 for accuracy.

def inv_optimus_fn(a, T, scale):   # Maps accuracy to time
    if a==0:
        return 0
    else:
        return scale*a/(10/T * (100-a*scale))

def linear_fn(t, k):
    return k*t

def inv_linear_fn(a, k):
    return a/k

def get_linear_fn(k):
    return lambda t: linear_fn(t, k), lambda a: inv_linear_fn(a, k)

def get_optimus_fn(T, scale):
    return lambda t: optimus_fn(t, T, scale), lambda a: inv_optimus_fn(a, T, scale)

def get_AUC_curves(train_models, run_schedule, T):
    # Train_models is a list of models in the order they are trained
    times_to_run_first, times_to_run_second = run_schedule 
    assert len(train_models) == 2
    first_model = train_models[0]
    second_model = train_models[1]
    x_data = list(range(0, T))
    acc1 = []   # First accuracy curve
    acc2 = []
    was_first_training, was_second_training = False, False
    for t in x_data:
        is_first_training = t in times_to_run_first
        is_second_training = t in times_to_run_second
        if is_first_training:
            # Model 1 is training
            first_model.train(1)
        elif was_first_training:
            first_model.checkpoint()
        if is_second_training:
            # Model 2 is training
            second_model.train(1)   
        elif was_second_training:
            second_model.checkpoint()         
        if is_first_training or is_second_training:
            acc1.append(slowed_acc(first_model.acc))
            acc2.append(slowed_acc(second_model.acc))
        else:
            acc1.append(first_model.acc)
            acc2.append(second_model.acc)
        was_first_training = is_first_training
        was_second_training = is_second_training
    assert len(acc1) == len(acc2) == len(x_data)
    return acc1, acc2, x_data

def get_deltacc_run_schedule(A, B, train_budget, T):
    times_to_run_A = []
    times_to_run_B = []
    for t in range(0,T):
        if len(times_to_run_A) + len(times_to_run_B) > train_budget:
            break
        a_delta_acc = A.delta_acc(t + len(times_to_run_A))
        b_delta_acc = B.delta_acc(t + len(times_to_run_B))
        if a_delta_acc > b_delta_acc:
            times_to_run_A.append(t)
        else:
            times_to_run_B.append(t)
        #print(a_delta_acc, b_delta_acc)
    return times_to_run_A, times_to_run_B

def plot_train_profiles(train_funcs, time_max=100, title="Accuracy vs GPU Cycles"):
    t_data_train = list(range(0,time_max))
    for fn, label in train_funcs:
        tacc = [fn(i) for i in t_data_train]
        plt.plot(t_data_train, tacc, label=label)
    plt.legend()
    plt.title(title)
    plt.xlabel("GPU Time")
    plt.ylabel("Accuracy")
    plt.ylim([0,100])

def plot_acc_data(lines, t_data, title="Accuracy over time"):
    for l in lines:
        acc_data, label = l
        plt.plot(t_data, acc_data, label=label)
    plt.legend()
    plt.title(title)
    plt.xlabel("Time")
    plt.ylabel("Accuracy")
    plt.ylim([0,100])

def plot_auc_plane(train_models, T):
    A,B = train_models
    train_budgets = list(range(0, T))
    auc_data = []
    runsched_list = []
    acc1_list = []
    acc2_list = []
    for budget in train_budgets:
        A.reset()
        B.reset()
        run_schedule = get_deltacc_run_schedule(A, B, budget, T)
        acc1, acc2, t_data = get_AUC_curves(train_models, run_schedule, T)
        auc = sum(acc1) + sum(acc2)
        auc_data.append(auc)
        acc1_list.append(acc1)
        acc2_list.append(acc2)
        runsched_list.append(run_schedule)
    
    max_index = auc_data.index(max(auc_data))
    budget_opt, acc1_opt, acc2_opt, auc_opt, runsched_opt = train_budgets[max_index], acc1_list[max_index], acc2_list[max_index], auc_data[max_index], runsched_list[max_index]
    print("Max coords: budget: {}, AUC: {}".format(budget_opt, auc_opt))
    print("Runsched: {}".format(runsched_opt))

    fig = plt.figure()
    ax = fig.gca()
    # Plot the surface.
    p = ax.scatter(train_budgets, auc_data)  # , linewidth=0, antialiased=False)
    ax.set_xlabel("TrainBudget")
    ax.set_ylabel("AUC")
    ax.set_title("Train Budget vs AUC")

    plt.figure()
    plot_acc_data([(acc1_opt, "FirstModel"), (acc2_opt, "SecondModel")], list(range(0, T)), title="Best Inference Accuracy over Time")
    print(sum(acc1_opt) + sum(acc2_opt))
    
#     plt.figure()
#     tacc1, tacc2, t_data_train = get_train_curves([A, B], t1_opt, t2_opt, T)
#     plot_acc_data([(tacc1, "FirstModel_Train"), (tacc2, "SecondModel_Train")], t_data_train, title="Best Train Accuracy over Time")
    
#     plt.figure()
#     plot_train_profiles([(A.train_fn, "A_Train_Profile"), (B.train_fn, "B_Train_Profile")])
                        
    plt.show()

### Simulator start

In [11]:
def run_sim(a_conv_time = 50, a_scale = 1, b_conv_time = 50, b_scale = 1, target_start_accuracy = 30, T = 20):
    a_func, a_inv_func = get_optimus_fn(a_conv_time, a_scale)
    b_func, b_inv_func = get_optimus_fn(b_conv_time, b_scale)
    init_time_a = a_inv_func(target_start_accuracy)
    init_time_b = b_inv_func(target_start_accuracy)
    A = Model(a_func, init_time_a, auto_checkpoint=False)
    B = Model(b_func, init_time_b, auto_checkpoint=False)

    plot_auc_plane([A,B], T)
    #plot_auc_plane([B,A], T)

In [12]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [13]:
interactive_plot = interactive(run_sim, {'manual': True}, a_conv_time = (0, 500, 1), a_scale = (0,1,0.1), b_conv_time = (0, 500, 1), b_scale = (0,1,0.1), target_start_accuracy = 40, T = 20)
output = interactive_plot.children[-1]
#output.layout.height = '1200px'
interactive_plot


interactive(children=(IntSlider(value=50, description='a_conv_time', max=500), FloatSlider(value=1.0, descript…